# **Introduction to Natural Language Processing 2 Lab02**

**1. Fine-tuning of the model on the training data**

In [1]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 31.7 MB/s 
     |████████████████████████████████| 182 kB 43.5 MB/s 
     |████████████████████████████████| 115 kB 66.5 MB/s 
     |████████████████████████████████| 212 kB 67.5 MB/s 
     |████████████████████████████████| 127 kB 68.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 28.6 MB/s 
     |████████████████████████████████| 7.6 MB 49.4 MB/s 


Connect to huggingface to get the pretrained model. You need a hugging face token to login.

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


Load IMDb dataset

In [3]:
from datasets import load_dataset, load_metric

data = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Load Pretrained DistilBERT model

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# We are defining our model we want to train. We choose distilbert-base-uncased
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
# We need a tokenizer to preprocess our data. Each model has is own tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Prepocess Data by tokenizing text

In [26]:
def tokenize(examples):
    """
        Cut a text in tokens and return token dictionnary dataset

        Parameters
        ----------
        examples : <class 'datasets.arrow_dataset.Batch'>

        Returns
        -------
        outputs : datasets.dataset_dict.DatasetDict
    """
    outputs = tokenizer(examples['text'], truncation=True)
    return outputs

tokenized_data = data.map(tokenize, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

The next step is to train the model.

In [6]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

In [7]:
import numpy as np

def compute_metrics(eval_preds):
    """
        Return accuracy on given predictions

        Parameters
        ----------
        eval_preds : transformers.trainer_utils.PredictionOutput

        Returns
        -------
        outputs : dict
    """  
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
#We need to set training parameters. We will train the model on only one epoch
training_args = TrainingArguments(num_train_epochs=1,
                                  output_dir="distilbert-imdb",
                                  push_to_hub=True,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  evaluation_strategy="epoch")

In [9]:
#DataCollator will form a batch by using a list of dataset elements as input.
data_collator = DataCollatorWithPadding(tokenizer)

#We can a Trainer by passing it all the objects constructed up to now
trainer = Trainer(model=model, tokenizer=tokenizer,
                  data_collator=data_collator,
                  args=training_args,
                  train_dataset=tokenized_data["train"],
                  eval_dataset=tokenized_data["test"],
                  compute_metrics=compute_metrics)

Cloning https://huggingface.co/mattmax/distilbert-imdb into local empty directory.


In [10]:
model.device

device(type='cuda', index=0)

In [11]:
# We are now starting the training
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1563
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster 

Epoch,Training Loss,Validation Loss,Accuracy
1,0.223700,0.186340,0.928320


Saving model checkpoint to distilbert-imdb/checkpoint-500
Configuration saved in distilbert-imdb/checkpoint-500/config.json
Model weights saved in distilbert-imdb/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbert-imdb/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbert-imdb/checkpoint-500/special_tokens_map.json
tokenizer config file saved in distilbert-imdb/tokenizer_config.json
Special tokens file saved in distilbert-imdb/special_tokens_map.json
Saving model checkpoint to distilbert-imdb/checkpoint-1000
Configuration saved in distilbert-imdb/checkpoint-1000/config.json
Model weights saved in distilbert-imdb/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in distilbert-imdb/checkpoint-1000/tokenizer_config.json
Special tokens file saved in distilbert-imdb/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to distilbert-imdb/checkpoint-1500
Configuration saved in distilbert-imdb/checkpoint-1500/config.json
Mod



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1563, training_loss=0.2592922742749664, metrics={'train_runtime': 1550.0262, 'train_samples_per_second': 16.129, 'train_steps_per_second': 1.008, 'total_flos': 3281068438885632.0, 'train_loss': 0.2592922742749664, 'epoch': 1.0})

Predict on test dataset

In [12]:
# We want to get predictions on test dataset
predictions = trainer.predict(tokenized_data["test"])

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 25000
  Batch size = 16


Get the test predictions accuracy

In [17]:
metric = load_metric('accuracy')
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.92832}

**2. We have an accuracy of 0.9286 for the test dataset.**

In [25]:
def get_wrong_classifications(max, preds, predictions, ds):
    """
        Look for wrong classified text and return a list of it

        Parameters
        ----------
        max : int
        preds : numpy.ndarray
        predictions : transformers.trainer_utils.PredictionOutput
        ds : datasets.dataset_dict.DatasetDict

        Returns
        -------
        wrongs : list
    """  
    count = 0
    wrongs = []
    for i in range(len(preds)):
      if predictions.label_ids[i] != preds[i]: 
        wrongs.append(ds["test"]["text"][i])
        count += 1
      if count == max:
        break
    return wrongs

get_wrong_classifications(2, preds, predictions, data)

["First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie. Most of these movies may not have the best plots or best actors but I enjoy these kinds of movies for what they are. This movie is much better than any of the movies the other action guys (Segal and Dolph) have thought about putting out the past few years. Van Damme is good in the movie, the movie is only worth watching to Van Damme fans. It is not as good as Wake of Death (which i highly recommend to anyone of likes Van Damme) or In hell but, in my opinion it's worth watching. It has the same type of feel to it as Nowhere to Run. Good fun stuff!",
 "Ben, (Rupert Grint), is a deeply unhappy adolescent, the son of his unhappily married parents. His father, (Nicholas Farrell), is a vicar and his mother, (Laura Linney), is ... well, let's just say she's a somewhat hypocritical soldier in Jesus' army. It's only when he takes a summer job as an assistant to a foul-mouthe

**3. For the first wrong classified comment, the review uses a lot of negative sentence to describe similar movies and previous movies with this actor, but he reveals afterwards that this one is an exception, so the comment is quite suble so it is hard for the model to get it right. Concerning the second one, he is describing at first the movie which is a really sad story with a lot of negative adjectives, but the author appreciated it. The model could not understand the difference between the plot of the movie and the review itself**

**4. The main advantage is that for this approach is it's efficiency. However it is way longer to train (as we can see we used only one epoch).**